In [1]:
# | default_exp agent.planner.task_generator

In [ ]:
# | export
import json
import re
from typing import Optional
from rich.console import Console
from datetime import datetime
from agentic.agent.planner.models import Task, ProjectBreakdown, ProjectContext


In [2]:
# | export 
class TaskGenerator:
    """Handles task generation and validation"""
    
    def __init__(self, agent, console: Console):
        self.agent = agent
        self.console = console
    
    def _determine_project_phase(self, completed: int, total: int) -> str:
        """Determine current project phase based on completion"""
        progress = completed / max(total, 1)
        if progress < 0.3:
            return "SETUP & FOUNDATION"
        elif progress < 0.6:
            return "CORE DEVELOPMENT"
        elif progress < 0.8:
            return "INTEGRATION & FEATURES"
        else:
            return "TESTING & FINALIZATION"
    
    def _get_phase_guidance(self, completed: int, total: int) -> str:
        """Get phase-specific guidance for task generation"""
        phase = self._determine_project_phase(completed, total)
        
        guidance = {
            "SETUP & FOUNDATION": """
• Focus: Project structure, requirements analysis, environment setup
• Priority: Create solid foundation for development
• Tasks: Directory structure, dependency management, configuration files
• Quality: Establish coding standards and project conventions""",
            
            "CORE DEVELOPMENT": """
• Focus: Main functionality implementation, core components
• Priority: Build primary features and business logic
• Tasks: Core modules, main algorithms, data structures
• Quality: Robust error handling and comprehensive logging""",
            
            "INTEGRATION & FEATURES": """
• Focus: Component integration, advanced features, optimization
• Priority: Connect systems and add sophisticated functionality
• Tasks: Module integration, advanced features, performance tuning
• Quality: Code optimization and maintainability""",
            
            "TESTING & FINALIZATION": """
• Focus: Quality assurance, documentation, final polish
• Priority: Ensure reliability and completeness
• Tasks: Testing (if required), documentation, final validation
• Quality: Code review and production readiness"""
        }
        
        return guidance.get(phase, "Continue with next logical development step")
    
    
    def generate_next_task(self, context: ProjectContext, breakdown: ProjectBreakdown, estimated_total: int) -> Optional[Task]:
        """Generate single next task using project breakdown context"""
        
        context_prompt = self._build_context_prompt(context)
        execution_context = self._get_execution_context(context)
        
        completed_count = len(context.execution_history)
        next_task_id = f"T{completed_count + 1:03d}"
        
        prompt = f"""
MISSION: Generate the next critical task in the software development pipeline.

EXECUTION CONTEXT:
• Progress: {completed_count}/{estimated_total} tasks completed
• Project Phase: {self._determine_project_phase(completed_count, estimated_total)}
• Quality Standard: Production-grade implementation required

{execution_context}

PROJECT SPECIFICATION:
• Objective: {breakdown.project_summary}
• Development Phases: {' → '.join(breakdown.phases)}
• Target Deliverables: {', '.join(breakdown.key_deliverables)}
• Technical Stack: {breakdown.technical_approach}

ORIGINAL REQUIREMENT: {context.original_request}

{context_prompt}

TASK GENERATION STRATEGY:
{self._get_phase_guidance(completed_count, estimated_total)}

RESPONSE FORMAT - SINGLE TASK OBJECT:
{{
  "id": "{next_task_id}",
  "name": "[Descriptive Task Name - Action Oriented]",
  "description": "[Detailed description aligned with current project phase and deliverables]",
  "dependencies": [/* Previous task IDs if applicable */],
  "actions": [
    {{
      "step": 1,
      "purpose": "[Clear, actionable purpose statement]",
      "sub_steps": [
        "[Specific action 1 - mention exact files/outputs]",
        "[Specific action 2 - mention exact files/outputs]",
        "[Specific action 3 - mention exact files/outputs]"
      ],
      "introspect_after": true,
      "system_prompt": "You are a senior software engineer working on: {breakdown.project_summary}. Deliver high-quality code with proper error handling, documentation, and best practices.",
      "user_prompt": "Implement the required functionality focusing on: [LIST ALL expected_outputs HERE]. Use appropriate tools (fs_write, execute_bash) to create well-structured, documented code.",
      "introspect_prompt": "Validate implementation quality: check file existence, code structure, documentation completeness, and adherence to requirements.",
      "execution_mode": "sequential"
    }}
  ],
  "success_criteria": "[Specific, measurable success criteria with file/component verification]",
  "expected_outputs": ["[specific_file1.py]", "[specific_file2.json]", "[directory/]"],
  "potential_options": [],
  "needs_debate": false
}}

DEBATE USAGE GUIDELINES:
• Set "needs_debate": true for tasks involving:
  - Architecture decisions (database choice, framework selection)
  - Technology stack decisions (library comparisons, tool selection)
  - Design pattern choices (authentication methods, API design)
  - Performance optimization approaches
• Set "potential_options": ["option1", "option2", "option3"] when debate is needed
• Use debate sparingly - only for significant decisions, not implementation details

QUALITY REQUIREMENTS:
✓ All expected_outputs must be explicitly mentioned in sub_steps and user_prompt
✓ Task must advance toward project objectives
✓ Actions must be specific and executable
✓ Success criteria must be measurable
✓ High-quality standards apply

Generate the next logical task or return null if project is complete.
"""
        
        try:
            response = self.agent.run(prompt).get("content", "")
            
            # Save raw response for debugging in JSONL format
            import json
            from datetime import datetime
            log_entry = {
                "timestamp": datetime.now().isoformat(),
                "task_id": next_task_id,
                "title": breakdown.title if hasattr(breakdown, 'title') else "Unknown Project",
                "response": response
            }
            with open("task_generation_raw.jsonl", 'a') as f:
                f.write(json.dumps(log_entry) + '\n')
            
            # Parse and validate task
            response = response.strip()
            
            # Only return None if explicitly null AND we have enough tasks
            if response.lower() == "null":
                if context.total_tasks_completed >= 5:
                    return None
                else:
                    self.console.print("⚠️ LLM returned null but project needs more tasks. Regenerating...")
                    return self._force_generate_next_task(context, breakdown, estimated_total)
            
            # Try to extract JSON from response
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                json_str = json_match.group()
                task_data = json.loads(json_str)
            else:
                task_data = json.loads(response)
                
        except json.JSONDecodeError as e:
            self.console.print(f"❌ JSON parsing failed: {e}")
            task_data = self._extract_task_from_text(response, context)
            if not task_data:
                self.console.print("❌ Failed to extract task from response")
                return None
        
        # If LLM returns array, take only the first task
        if isinstance(task_data, list):
            if len(task_data) > 0:
                task_data = task_data[0]
                self.console.print("⚠️ LLM returned multiple tasks, using only the first one")
            else:
                self.console.print("⚠️ LLM returned empty task array")
                return None
        
        # Validate required fields
        if not isinstance(task_data, dict) or 'name' not in task_data:
            self.console.print("⚠️ Invalid task format returned by LLM")
            return None
        
        task = Task(**task_data)
        task.id = f"T{len(context.execution_history) + 1:03d}"
        
        # Check for task repetition
        for prev_task in context.execution_history:
            if task.name.lower().strip() == prev_task.task_name.lower().strip():
                self.console.print(f"⚠️ Task '{task.name}' appears to be a repeat. Skipping.")
                return None
        
        return task
    
    def regenerate_task_with_feedback(self, context: ProjectContext, breakdown: ProjectBreakdown, feedback: str) -> Optional[Task]:
        """Regenerate task incorporating introspection feedback"""
        context_prompt = self._build_context_prompt(context)
        execution_context = self._get_execution_context(context)
        
        prompt = f"""
CRITICAL: The previously generated task was rejected. Generate a BETTER task incorporating the feedback.

FEEDBACK FROM VALIDATION: {feedback}

{execution_context}

PROJECT BREAKDOWN CONTEXT:
Summary: {breakdown.project_summary}
Phases: {', '.join(breakdown.phases)}
Key Deliverables: {', '.join(breakdown.key_deliverables)}

{context_prompt}

Generate ONE improved task that addresses the validation feedback. Ensure it's unique and not a repeat.

RESPOND WITH SINGLE TASK OBJECT OR NULL:
"""
        
        try:
            agent_response = self.agent.run(prompt)
            response = agent_response.get("content", "") if isinstance(agent_response, dict) else str(agent_response)
            
            if response.strip().lower() == "null":
                return None
                
            task_data = json.loads(response.strip())
            
        except json.JSONDecodeError:
            task_data = self._extract_task_from_text(response.strip(), context)
            if not task_data:
                raise ValueError("Failed to parse JSON and extract task fields. Please provide valid JSON format with required fields: id, name, description, actions.")
            
            if isinstance(task_data, list) and len(task_data) > 0:
                task_data = task_data[0]
            elif isinstance(task_data, list):
                return None
                
            if not isinstance(task_data, dict) or 'name' not in task_data:
                return None
                
            task = Task(**task_data)
            task.id = f"T{len(context.execution_history) + 1:03d}"
            
            return task
            
        except Exception as e:
            self.console.print(f"❌ Task regeneration failed: {e}")
            return None
    
    def _force_generate_next_task(self, context: ProjectContext, breakdown: ProjectBreakdown, estimated_total: int) -> Optional[Task]:
        """Force generation of next task when LLM returns null prematurely"""
        
        completed_count = len(context.execution_history)
        next_task_id = f"T{completed_count + 1:03d}"
        
        # Determine what type of task is needed based on progress
        if completed_count < 3:
            task_type = "setup and structure"
            example_tasks = ["Create project structure", "Setup dependencies", "Configure environment"]
        elif completed_count < 6:
            task_type = "core implementation"
            example_tasks = ["Implement main scraper", "Add rate limiting", "Create data models"]
        else:
            task_type = "integration and testing"
            example_tasks = ["Add error handling", "Create tests", "Setup deployment"]
        
        prompt = f"""
CRITICAL: The project needs more tasks to be complete. You MUST generate a task.

CURRENT STATUS: Only {completed_count} tasks completed - this is insufficient for a complete web scraping framework.

REQUIRED TASK TYPE: {task_type}
EXAMPLES: {', '.join(example_tasks)}

PROJECT CONTEXT:
- Original Request: {context.original_request}
- Project Summary: {breakdown.project_summary}
- Completed Tasks: {[t.task_name for t in context.execution_history]}

Generate a {task_type} task that is essential for completing the web scraping framework.

RESPOND WITH TASK OBJECT:
{{
  "id": "{next_task_id}",
  "name": "Essential Task Name",
  "description": "Description of essential task",
  "dependencies": [],
  "actions": [
    {{
      "step": 1,
      "purpose": "Main purpose",
      "sub_steps": ["Step 1", "Step 2"],
      "introspect_after": true,
      "system_prompt": "You are a developer working on a web scraping framework. Complete this essential task.",
      "user_prompt": "Implement the required functionality for the web scraping framework.",
      "introspect_prompt": "Validate that the implementation is correct and functional.",
      "execution_mode": "sequential"
    }}
  ],
  "success_criteria": "Task completion criteria",
  "expected_outputs": ["output.py"],
  "potential_options": [],
  "needs_debate": false
}}
"""
        
        try:
            response = self.agent.run(prompt).get("content", "")
            
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                task_data = json.loads(json_match.group())
            else:
                task_data = json.loads(response.strip())
            
            if isinstance(task_data, dict) and 'name' in task_data:
                task = Task(**task_data)
                task.id = next_task_id
                return task
                
        except Exception as e:
            self.console.print(f"❌ Force task generation failed: {e}")
        
        return None
    
    def _build_context_prompt(self, context: ProjectContext) -> str:
        """Build comprehensive context from execution history"""
        if not context.execution_history:
            return "PREVIOUS CONTEXT: This is the first task."
        
        context_parts = ["EXECUTION HISTORY:"]
        
        for i, task_result in enumerate(context.execution_history, 1):
            status_emoji = "✅" if task_result.status == "success" else "❌"
            context_parts.append(f"{i}. {task_result.task_name} (ID: {task_result.task_id}) - {status_emoji} {task_result.status}")
            context_parts.append(f"   Description: {task_result.task_name}")
            if task_result.artifacts_created:
                if len(task_result.artifacts_created) <= 3:
                    context_parts.append(f"   Artifacts: {', '.join(task_result.artifacts_created)}")
                else:
                    shown = ', '.join(task_result.artifacts_created[:3])
                    remaining = len(task_result.artifacts_created) - 3
                    context_parts.append(f"   Artifacts: {shown} ... (+{remaining} more)")
        
        context_parts.append(f"\nTOTAL ARTIFACTS: {len(context.current_artifacts)}")
        context_parts.append(f"PROJECT STATUS: {context.project_status}")
        context_parts.append(f"PROGRESS: {context.total_tasks_completed}/{len(context.execution_history)} tasks completed")
        
        return "\n".join(context_parts)
    
    def _get_execution_context(self, context: ProjectContext) -> str:
        """Generate minimal context for next task generation"""
        if not context.execution_history:
            return "CONTEXT: This is the first task."
        
        last_task = context.execution_history[-1].task_name
        artifacts_count = len(context.current_artifacts)
        
        return f"CONTEXT: Last completed task: '{last_task}'. Total artifacts created: {artifacts_count}."
    
    def _extract_task_from_text(self, text: str, context: ProjectContext) -> Optional[dict]:
        """Extract task fields from malformed JSON/text as fallback"""
        
        json_match = re.search(r'\{.*\}', text, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group())
            except:
                pass
        
        task_data = {}
        
        if match := re.search(r'"?id"?\s*:\s*"?([^",\n]+)"?', text, re.IGNORECASE):
            task_data["id"] = match.group(1).strip('"')
        
        if match := re.search(r'"?name"?\s*:\s*"([^"]+)"', text, re.IGNORECASE):
            task_data["name"] = match.group(1)
        
        if match := re.search(r'"?description"?\s*:\s*"([^"]+)"', text, re.IGNORECASE):
            task_data["description"] = match.group(1)
        
        if "name" in task_data:
            task_data.setdefault("id", f"T{len(context.execution_history) + 1:03d}")
            task_data.setdefault("description", task_data["name"])
            task_data.setdefault("dependencies", [])
            task_data.setdefault("success_criteria", f"Complete {task_data['name']}")
            task_data.setdefault("expected_outputs", [])
            task_data.setdefault("potential_options", [])
            task_data.setdefault("needs_debate", False)
            
            task_data.setdefault("actions", [{
                "step": 1,
                "purpose": task_data["name"],
                "sub_steps": [f"Create {output}" for output in task_data.get("expected_outputs", [])],
                "introspect_after": True,
                "system_prompt": f"You are an AI assistant. Complete the task: {task_data['name']}. Create all expected outputs.",
                "user_prompt": f"{task_data['description']}. Create all expected outputs using fs_write tool: {', '.join(task_data.get('expected_outputs', []))}",
                "introspect_prompt": f"Validate that {task_data['name']} was completed successfully and all expected outputs exist",
                "execution_mode": "sequential"
            }])
            
            return task_data
        
        return None



NameError: name 'Console' is not defined